In [ ]:
from bs4 import BeatifulSoup as bs

In [1]:
def get_web_page(url):
    resp=request.get(
        url=url,
        cookies={'over18':'1'})
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [2]:
def get_articles(dom, date):
    soup = bs(dom,'html5lib')
    
    #Get previous page link
    paging_div = soup.find('div','btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']
    
    articles = []
    divs = soup.find_all('div','r-ent')
    for d in divs:
        if d.find('div', 'date').text.strip() == date:
            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)
                except ValueError:
                    # if transfer failure maybe is '爆', 'x1', 'x2'
                    # if don`t do anything push_count keep 0
                    if push_str == '爆':
                        push_count == 99
                    elif push_str.startswith('X'):
                        push_count = -10
        if d.find('a'): #Have hyperLink means paragraph exist else delete
            href = d.find('a')['href']
            title = d.find('a').text
            author = d.find('div', 'author').text if d.find('div', 'author') else ''
            articles.append({
                'title' : title,
                'href' : href,
                'push_count' : push_count,
                'author' : author
            })
    return articles, prev_url

In [ ]:
ppt_url = 'https://www.ptt.cc/bbs'
current_page = get_web_page(ppt_url + '/bbs/Gossiping/index.html')